In [18]:
from unidecode import unidecode
import pandas as pd
from datetime import datetime

# --- 1. Cargar los datos ---
df = pd.read_csv("resultados.csv",sep=";",encoding="utf-8", dtype=str)
df.head()

,periodo,estu_tipodocumento,estu_consecutivo,cole_area_ubicacion,cole_bilingue,cole_calendario,cole_caracter,cole_cod_dane_establecimiento,cole_cod_dane_sede,cole_cod_depto_ubicacion,...,fami_tienecomputador,fami_tieneinternet,fami_tienelavadora,desemp_ingles,punt_ingles,punt_matematicas,punt_sociales_ciudadanas,punt_c_naturales,punt_lectura_critica,punt_global
0,20172,TI,SB11201720520268,URBANO,N,A,ACADÉMICO,"1,41298E+11","1,41298E+11",41,...,No,No,No,A-,31.0,31.0,31,40,41,177
1,20172,TI,SB11201720355240,URBANO,N,A,TÉCNICO,"1,25572E+11","1,25572E+11",25,...,No,No,No,A1,48.0,64.0,47,58,61,284
2,20172,TI,SB11201720447341,URBANO,NaN,A,ACADÉMICO,"1,05284E+11","1,05284E+11",5,...,No,No,No,A-,43.0,38.0,49,52,58,244
3,20172,TI,SB11201720000089,URBANO,N,A,ACADÉMICO,"4,0538E+11","4,0538E+11",5,...,Si,Si,Si,A1,52.0,63.0,66,63,66,318
4,20172,CC,SB11201720000685,RURAL,N,A,ACADÉMICO,"4,0544E+11","4,0544E+11",5,...,No,Si,Si,B1,76.0,69.0,71,68,64,343


In [3]:
from unidecode import unidecode

# Reparar y limpiar acentos solo en strings
def reparar_y_normalizar(texto):
    if isinstance(texto, str):
        try:
            texto_reparado = texto.encode('latin1').decode('utf-8')
        except:
            texto_reparado = texto
        return unidecode(texto_reparado)
    return texto

df['estu_depto_reside'] = df['estu_depto_reside'].apply(reparar_y_normalizar)
df['estu_mcpio_reside'] = df['estu_mcpio_reside'].apply(reparar_y_normalizar)


In [4]:
# Convertir estu_fechanacimiento a datetime
df['estu_fechanacimiento'] = pd.to_datetime(df['estu_fechanacimiento'], format='mixed', dayfirst=True, errors='coerce')

# Crear columna de fecha del examen
df['fecha_examen'] = df['periodo'].astype(int).map({
    20171: "2017-09-01",
    20172: "2017-03-01"
})
df['fecha_examen'] = pd.to_datetime(df['fecha_examen'])  # Convertir explícitamente a datetime

# Calcular edad
df['edad'] = ((df['fecha_examen'] - df['estu_fechanacimiento']).dt.days // 365)


In [5]:
# --- 4. Mapear variables binarias ---
binarias = ['fami_tieneautomovil', 'fami_tienecomputador', 'fami_tieneinternet', 'fami_tienelavadora']
for col in binarias:
    df[col] = df[col].map({'Si': 1, 'No': 0})

In [6]:
# Asegurar que la columna sea tipo string
df['fami_estratovivienda'] = df['fami_estratovivienda'].astype(str)

# Extraer solo el número del estrato
df['fami_estratovivienda'] = df['fami_estratovivienda'].str.extract(r'(\d+)')

# Convertir a numérico (int o float), permitiendo valores nulos
df['fami_estratovivienda'] = pd.to_numeric(df['fami_estratovivienda'], errors='coerce')


In [7]:
# --- 6. Convertir puntaje global ---
df['punt_global'] = pd.to_numeric(df['punt_global'])


In [8]:
# --- 8. Rellenar valores faltantes ---
binarias_a_completar = ['fami_tieneautomovil', 'fami_tienecomputador', 'fami_tieneinternet', 'fami_tienelavadora']
categoricas = ['estu_depto_reside', 'estu_mcpio_reside', 'estu_nacionalidad']
numericas = ['edad', 'fami_estratovivienda', 'punt_global']

In [9]:
# Binarias (rellenar con moda si existe)
for col in binarias_a_completar:
    moda = df[col].mode(dropna=True)
    if not moda.empty:
        df[col] = df[col].fillna(moda[0])
    else:
        df[col] = df[col].fillna(0)  # Valor por defecto si no hay moda

# Categóricas (rellenar con moda si existe)
for col in categoricas:
    moda = df[col].mode(dropna=True)
    if not moda.empty:
        df[col] = df[col].fillna(moda[0])
    else:
        df[col] = df[col].fillna("No especificado")

# Numéricas (rellenar con mediana si existe)
for col in numericas:
    if df[col].notnull().any():
        df[col] = df[col].fillna(df[col].median())
    else:
        df[col] = df[col].fillna(0)


In [10]:
# --- 9. Calcular proxy de SISBÉN ---
df['sisben_score'] = (
    (df['fami_estratovivienda'] <= 2).astype(int) +
    (df['fami_tienecomputador'] == 0).astype(int) +
    (df['fami_tieneinternet'] == 0).astype(int) +
    (df['fami_tieneautomovil'] == 0).astype(int) +
    (df['fami_tienelavadora'] == 0).astype(int)
)


In [11]:
# --- 9. Calcular proxy de SISBÉN ---
df['sisben_score'] = (
    (df['fami_estratovivienda'] <= 2).astype(int) +
    (df['fami_tienecomputador'] == 0).astype(int) +
    (df['fami_tieneinternet'] == 0).astype(int) +
    (df['fami_tieneautomovil'] == 0).astype(int) +
    (df['fami_tienelavadora'] == 0).astype(int)
)
df['es_sisben_proxy'] = (df['sisben_score'] >= 3).astype(int)

In [16]:
# --- 10. Exportar a Excel o CSV ---
# Crear variable binaria de elegibilidad
df['es_elegible_jovenes_u'] = (
    (df['edad'] < 28) &
    (df['estu_depto_reside'].str.upper().str.strip() == 'BOGOTA') &
    (df['es_sisben_proxy'] == 1)
).astype(int)

columnas_utiles = [
    'periodo',                  # Identificador de la convocatoria
    'edad',                     # Variable demográfica
    'estu_nacionalidad',        # Variable demográfica
    'estu_depto_reside',        # Ubicación geográfica
    'estu_mcpio_reside',        # Ubicación geográfica
    'fami_estratovivienda',     # Nivel socioeconómico
    'fami_tieneautomovil',      # Condición socioeconómica
    'fami_tienecomputador',     # Condición socioeconómica
    'fami_tieneinternet',       # Condición socioeconómica
    'fami_tienelavadora',       # Condición socioeconómica
    'es_sisben_proxy',          # Etiqueta binaria basada en condiciones socioeconómicas
    'punt_global',               # Resultado académico
    'es_elegible_jovenes_u'     #Variable de interes
]

df=df[columnas_utiles]
df.head()



,periodo,edad,estu_nacionalidad,estu_depto_reside,estu_mcpio_reside,fami_estratovivienda,fami_tieneautomovil,fami_tienecomputador,fami_tieneinternet,fami_tienelavadora,es_sisben_proxy,punt_global,es_elegible_jovenes_u
0,20172,17.0,COLOMBIA,HUILA,GARZON,1.0,0.0,0.0,0.0,0.0,1,177,0
1,20172,16.0,COLOMBIA,CUNDINAMARCA,PUERTO SALGAR,1.0,0.0,0.0,0.0,0.0,1,284,0
2,20172,15.0,COLOMBIA,ANTIOQUIA,FRONTINO,2.0,0.0,0.0,0.0,0.0,1,244,0
3,20172,16.0,COLOMBIA,ANTIOQUIA,ITAGUI,3.0,0.0,1.0,1.0,1.0,0,318,0
4,20172,18.0,COLOMBIA,ANTIOQUIA,MARINILLA,3.0,0.0,0.0,1.0,1.0,0,343,0


In [17]:

# Exportar a Excel
df.to_excel("datos_utiles.xlsx", index=False)

# O si prefieres CSV
# df_util.to_csv("datos_utiles.csv", index=False, encoding="utf-8-sig")

print("✅ Datos útiles exportados correctamente a 'datos_utiles.xlsx'")


✅ Datos útiles exportados correctamente a 'datos_utiles.xlsx'
